In [1]:
pip install --upgrade nbconvert nbformat ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


# Практическая работа: Разработка системы генерации кода на основе архитектуры Code-T5

**Автор:** Егорычев Алексей

## Аннотация

В этой практической работе реализована воспроизводимая и интерактивная
Google Colab-ноутбук-версия проекта по созданию системы генерации кода
на базе архитектуры **CodeT5** (Salesforce). Для демонстрации и
оптимизации под ограничения Google Colab используется облегчённая
конфигурация: небольшая подвыборка данных (200 примеров) и компактная
модель **codet5-small**. Работа включает теорию, подготовку данных,
реализацию модели, обучение, оценку и визуализацию результатов.

# 1. УСТАНОВКА И ИМПОРТ БИБЛИОТЕК

In [1]:
# @title Установка необходимых библиотек
!pip install -q transformers datasets accelerate sentencepiece sacrebleu evaluate plotly tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


In [2]:
# @title Импорт основных библиотек
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments)
import evaluate
import json

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print('torch:', torch.__version__)

torch: 2.8.0+cu126


# 2. ТЕОРЕТИЧЕСКАЯ ЧАСТЬ




### Проблема
Автоматизация программирования — задача генерации корректных фрагментов кода по формулировке на естественном языке.

### Основные концепции
- **Seq2Seq Transformer**: CodeT5-small использует архитектуру encoder-decoder.
- **Tokenization**: SentencePiece для кода.
- **Optimization**: кросс-энтропия, авто-регрессивная генерация.

### Математическое обоснование
Модель обучается максимизировать ∑ log P(y_t | y_<t, x) при входном описании задачи x.

### Схема архитектуры
Вход (NL) → Encoder → Decoder → Код.


# 3. ПОДГОТОВКА ДАННЫХ

In [1]:
!wget https://storage.yandexcloud.net/img-to-bot/codesearchnet_python_sample.json

--2025-09-23 09:24:25--  https://storage.yandexcloud.net/img-to-bot/codesearchnet_python_sample.json
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27722 (27K) [application/json]
Saving to: ‘codesearchnet_python_sample.json’

codesearchnet_pytho 100%[===================>]  27.07K  --.-KB/s    in 0.1s    

2025-09-23 09:24:26 (215 KB/s) - ‘codesearchnet_python_sample.json’ saved [27722/27722]



In [3]:
# @title Загрузка подготовленного JSON {display-mode: "form"}
DATA_PATH = "codesearchnet_python_sample.json"  # @param {type:"string"}

with open(DATA_PATH, "r") as f:
    data = json.load(f)

# создаём датасет HF
ds = Dataset.from_list(data)

print("Всего примеров:", len(ds))
print(ds[0])
# Разделение на train/val/test
split = ds.train_test_split(test_size=0.2, seed=RANDOM_SEED)
train_val = split['train'].train_test_split(test_size=0.25, seed=RANDOM_SEED)
train_ds, val_ds = train_val['train'], train_val['test']
test_ds = split['test']
print('train:', len(train_ds), 'val:', len(val_ds), 'test:', len(test_ds))

Всего примеров: 200
{'docstring': 'Return the sum of two numbers.', 'code': 'def add(a, b):\n    return a + b'}
train: 120 val: 40 test: 40


In [5]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=89e1ed9330ed78acf651202bbfc5d6a3528245e89a1116693482e37f4f6193d0
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


# 4. РЕАЛИЗАЦИЯ МОДЕЛИ

In [15]:
# @title Параметры модели {display-mode: "form"}
MODEL_NAME = "Salesforce/codet5-small"  # @param {type: 'string'}
MAX_INPUT_LENGTH = 128  # @param {type: 'integer'}
MAX_TARGET_LENGTH = 128  # @param {type: 'integer'}
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_for_model(batch):
    inputs = [x for x in batch['docstring']]
    targets = [x for x in batch['code']]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=MAX_TARGET_LENGTH, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_tok = train_ds.map(preprocess_for_model, batched=True, remove_columns=train_ds.column_names)
val_tok = val_ds.map(preprocess_for_model, batched=True, remove_columns=val_ds.column_names)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
bleu = evaluate.load('sacrebleu')
rouge = evaluate.load('rouge')

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bleu_score = bleu.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {'bleu': bleu_score['score'], 'rouge1': rouge_score['rouge1'], 'rougeL': rouge_score['rougeL']}
training_args = Seq2SeqTrainingArguments(
    output_dir='./codet5-small-exp',
    #evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir='./logs',
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

/tmp/ipython-input-4185848673.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# 5. ОБУЧЕНИЕ

In [16]:
# @title Запуск обучения {display-mode: "form"}
import os
os.environ["WANDB_DISABLED"] = "true"
START_TRAIN = True  # @param {type:"boolean"}
if START_TRAIN:
    trainer.train()
    trainer.save_model('./codet5-small-exp')

Step,Training Loss


# 6. ОЦЕНКА РЕЗУЛЬТАТОВ

In [17]:
prompts = [
    "Sort a list of integers in ascending order.",
    "Compute factorial of a number n using recursion.",
]
inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True).to(model.device)
outputs = model.generate(**inputs, max_length=64, num_beams=4)
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for p, d in zip(prompts, decoded):
    print('PROMPT:', p)
    print('GENERATED:\n', d)
    print('-'*40)
test_tok = test_ds.map(preprocess_for_model, batched=True, remove_columns=test_ds.column_names)
res = trainer.predict(test_tok)

# безопасное декодирование
preds = tokenizer.batch_decode(res.predictions, skip_special_tokens=True)
labels = np.where(res.label_ids != -100, res.label_ids, tokenizer.pad_token_id)
refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

print('Пример предсказаний:')
for p, r in zip(preds[:3], refs[:3]):
    print("PRED:", p)
    print("REF :", r)
    print('-'*40)

print('Metrics:', res.metrics)

PROMPT: Sort a list of integers in ascending order.
GENERATED:
 def sort(arr):
    return sorted(arr)
----------------------------------------
PROMPT: Compute factorial of a number n using recursion.
GENERATED:
 def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)
----------------------------------------


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Пример предсказаний:
PRED: def is_palindrome(s):
    return s == s[::-1
REF : def is_palindrome(s):
    return s == s[::-1]
----------------------------------------
PRED: def factorial(n):
    if n == 0:
        return 1
   
REF : def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)
----------------------------------------
PRED: def add(a, b):
    return a + b
REF : def add(a, b):
    return a + b
----------------------------------------
Metrics: {'test_loss': 0.0004237013345118612, 'test_bleu': 75.55651014799727, 'test_rouge1': 0.8784090909090909, 'test_rougeL': 0.8768398268398269, 'test_runtime': 29.8046, 'test_samples_per_second': 1.342, 'test_steps_per_second': 0.336}


# 7. ВЫВОДЫ




### Основные результаты
- Модель **CodeT5-small** была успешно обучена на искусственном датасете из 200 примеров.
- Система смогла сгенерировать корректные Python-функции по текстовым описаниям (например, сортировка списка, вычисление факториала).
- Получены высокие метрики на тестовой выборке:
  - **BLEU ≈ 75.6**
  - **ROUGE-L ≈ 0.87**
  - **Loss ≈ 0.0004**

### Анализ ошибок и особенностей
- Несмотря на высокие показатели, часть функций всё ещё сгенерирована неполностью (например, недостающие скобки).
- Модель переобучилась на маленьком датасете: она воспроизводит почти дословно тренировочные примеры.
- Реальная обобщающая способность остаётся ограниченной, так как датасет был слишком мал и однообразен.

### Проблемы и ограничения
- Использовалась компактная версия модели (**codet5-small**), которая имеет ограниченные возможности по сравнению с большими вариантами.
- Датасет был искусственно создан и слишком мал для настоящей генерации программного кода.
- Обучение проводилось всего за пять эпох, чтобы уложиться в лимиты Google Colab.

### Возможные улучшения
1. Использовать более крупные и разнообразные датасеты (например, полный CodeSearchNet).
2. Применить более мощные модели (CodeT5-base, CodeT5-large).
3. Увеличить число эпох обучения и применить регуляризацию, чтобы избежать переобучения.
4. Добавить более продвинутые метрики (CodeBLEU, функциональные тесты с запуском кода).
5. Визуализировать attention-карты, чтобы исследовать, как модель соотносит части описания и кода.

---

**Заключение:**  
Даже в сильно упрощённых условиях Google Colab удалось продемонстрировать, что архитектура **CodeT5** способна решать задачу генерации кода. Полученные результаты показывают высокую точность на игрушечном датасете, однако в реальных сценариях потребуется больше данных и ресурсов для достижения практического качества.